* pytorch에서 제공하는 torch.utils.data의 Dataset 클래스를 통해 데이터셋 만듬.
<br> 이때 아래 두 가지 함수 필수 구현 필요
    * __len__(): 함수를 통해 데이터의 크기 반환
    * __getitem__(): i번째 데이터(dataset[i])를 반환

In [13]:
import torch
import numpy as np
from torch.utils.data import Dataset
import librosa
from glob import glob
import random
import IPython.display as ipd


# 음성 파일의 sample rate은 1초 = 16000으로 지정한다
SR = 16000

In [12]:
## np.pad test
x = np.asarray([1,2,3,4,5,6])
np.pad(x,(5,3),'constant')

array([0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 0, 0, 0])

In [17]:
# 소음 데이터 확인
x = "../data/train/audio/_background_noise_/exercise_bike.wav"
data = librosa.load(x, sr=SR)
ipd.Audio(data[0], rate=data[1])
len(data[0])

980062

In [49]:
# 경진대회 전용 SpeechDataset 클래스를 정의한다
class SpeechDataset(Dataset):
    def __init__(self, mode, label_to_int, wav_list, label_list=None):
        self.mode = mode
        self.label_to_int = label_to_int
        self.wav_list = wav_list
        self.label_list = label_list
        self.sr = SR
        self.n_silence = int(len(wav_list) * 0.1) #  silence학습을 위해 10%의 무음 데이터 생성

        # 배경 소음 데이터를 미리 읽어온다
        self.background_noises = [librosa.load(x, sr=self.sr)[0] for x in glob("../data/train/audio/_background_noise_/*.wav")]

    def get_one_word_wav(self, idx):
        # idx 번째 음성 파일을 1초만큼 읽어온다
        wav = librosa.load(self.wav_list[idx], sr=self.sr)[0]
        if len(wav) < self.sr: ## 음성 길이가 sample rate보다 짧은 경우, 부족한 길이만큼 padding 
            wav = np.pad(wav, (0, self.sr - len(wav)), 'constant')
        return wav[:self.sr]

    def get_one_noise(self):
        # 배경 소음 데이터 중 랜덤하게 1초를 읽어온다
        selected_noise = self.background_noises[random.randint(0, len(self.background_noises) - 1)]
        start_idx = random.randint(0, len(selected_noise) - 1 - self.sr)
        return selected_noise[start_idx:(start_idx + self.sr)]

    def get_mix_noises(self, num_noise=1, max_ratio=0.1): ##?????????
        # num_noise 만큼의 배경 소음을 합성한다
        result = np.zeros(self.sr)
        for _ in range(num_noise):
            result += random.random() * max_ratio * self.get_one_noise()
        return result / num_noise if num_noise > 0 else result

    def get_silent_wav(self, num_noise=1, max_ratio=0.5):
        # 배경 소음 데이터를 silence로 가정하고 불러온다
        return self.get_mix_noises(num_noise=num_noise, max_ratio=max_ratio)

    def __len__(self):
        # 교차검증 모드일 경우에는 ‘silence’를 추가한 만큼이 데이터 크기이고, Test 모드일 경우에는 제공된 테스트 데이터가 전부이다
        if self.mode == 'test':
            return len(self.wav_list)
        else:
            return len(self.wav_list) + self.n_silence

    def __getitem__(self, idx):
        # idx번째 음성 데이터 하나를 반환한다
        if idx < len(self.wav_list):
            # 전처리는 mel spectrogram으로 지정한다
            # (옵션) 여기서 Data Augmentation을 수행할 수 있다.
            wav_numpy = preprocess_mel(self.get_one_word_wav(idx))
            wav_tensor = torch.from_numpy(wav_numpy).float()
            wav_tensor = wav_tensor.unsqueeze(0)

            # 음성 스펙트로그램(spec), 파일 경로(id)와 정답값(label)을 반환한다
            if self.mode == 'test':
                return {'spec': wav_tensor, 'id': self.wav_list[idx]}
            else:
                label = self.label_to_int.get(self.label_list[idx], len(self.label_to_int))
                return {'spec': wav_tensor, 'id': self.wav_list[idx], 'label': label}
        else:
            # 배경 소음을 반환한다
            wav_numpy = preprocess_mel(self.get_silent_wav(
                num_noise=random.choice([0, 1, 2, 3]),
                max_ratio=random.choice([x / 10. for x in range(20)])))
            wav_tensor = torch.from_numpy(wav_numpy).float()
            wav_tensor = wav_tensor.unsqueeze(0)
            return {'spec': wav_tensor, 'id': 'silence', 'label': len(self.label_to_int) + 1}

In [41]:
# mel spectrogram 전처리 함수이다
def preprocess_mel(data, n_mels=40):
    spectrogram = librosa.feature.melspectrogram(data, sr=SR, n_mels=n_mels, hop_length=160, \
                                                 n_fft=480, fmin=20, fmax=4000)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

#### 함수 Test

In [19]:
def get_one_word_wav(wav_list, idx):
    # idx 번째 음성 파일을 1초만큼 읽어온다
    wav = librosa.load(wav_list[idx], sr=SR)[0]
    if len(wav) < SR: ## 음성 길이가 sample rate보다 짧은 경우, 부족한 길이만큼 padding 
        wav = np.pad(wav, (0, SR - len(wav)), 'constant')
    return wav[:SR]

In [36]:
f = open('./input/trn.txt')
wav_list = []

for line in f.readlines():
#     print(line)
    wav = line.split(',')[2].split('\n')[0]
    wav_list.append(wav)
    

In [40]:
test_wav = get_one_word_wav(wav_list, 0)
test_wav.shape

(16000,)

In [42]:
wav_numpy = preprocess_mel(test_wav)
wav_numpy.shape

(40, 101)

In [50]:
background_noises = [librosa.load(x, sr=SR)[0] for x in glob("../data/train/audio/_background_noise_/*.wav")]



In [47]:
    def get_one_noise():
        # 배경 소음 데이터 중 랜덤하게 1초를 읽어온다
        selected_noise = background_noises[random.randint(0, len(background_noises) - 1)]
        start_idx = random.randint(0, len(selected_noise) - 1 - SR)
        return selected_noise[start_idx:(start_idx + SR)]



In [48]:
get_one_noise()

array([-0.41488647,  0.22940063, -0.05651855, ...,  0.47747803,
        0.15432739,  0.2833252 ], dtype=float32)